In [8]:
!jupyter nbextension enable --py widgetsnbextension


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [9]:
pip install jupyterlab-widgets


Note: you may need to restart the kernel to use updated packages.


In [12]:
!jupyter nbextension list


Known nbextensions:
  config dir: /Users/nethrashri/.jupyter/nbconfig
    notebook section
      jupyter-js-widgets/extension  enabled 
      - Validating: OK
  config dir: /Users/nethrashri/anaconda3/etc/jupyter/nbconfig
    notebook section
      jupyterlab-plotly/extension  enabled 
      - Validating: OK
      pydeck/extension  enabled 
      - Validating: problems found:
        - require?  X pydeck/extension
      jupyter-js-widgets/extension  enabled 
      - Validating: OK


In [13]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [7]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create a file upload button
upload_button = widgets.FileUpload(accept='image/*', multiple=False)
output_area = widgets.Output()

def on_upload_change(change):
    with output_area:
        clear_output()
        print("Uploaded file structure:")
        # Display the structure of the uploaded file data
        print(upload_button.value)

# Register the callback for the upload button
upload_button.observe(on_upload_change, names='value')

# Display the upload button and output area
display(upload_button, output_area)


FileUpload(value=(), accept='image/*', description='Upload')

Output()

In [8]:
import cv2
from fer import FER
import numpy as np
from collections import Counter
from IPython.display import display, Image, clear_output
import PIL.Image
import io
import ipywidgets as widgets
import json

# Initialize FER emotion detector
emotion_detector = FER(mtcnn=True)

# Widgets for file upload and processing
upload_button = widgets.FileUpload(accept='image/*', multiple=False)
submit_button = widgets.Button(description='Submit')
output_area = widgets.Output()

uploaded_file = None

def analyze_image_emotions(image_file_content):
    print("Starting emotion analysis...")
    emotion_sums = {
        'angry': 0,
        'disgust': 0,
        'fear': 0,
        'happy': 0,
        'sad': 0,
        'surprise': 0,
        'neutral': 0
    }
    all_emotions = []

    # Convert the uploaded file content to an image
    image_stream = io.BytesIO(image_file_content)
    image_pil = PIL.Image.open(image_stream)
    image_np = np.array(image_pil)

    # Convert the image from RGB (PIL) to BGR (OpenCV)
    image_bgr = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)

    # Resize the image to a consistent size for analysis
    height, width = image_bgr.shape[:2]
    aspect_ratio = width / height
    new_height = 480
    new_width = int(new_height * aspect_ratio)
    image_resized = cv2.resize(image_bgr, (new_width, new_height))

    # Analyze the emotions in the image
    emotions = emotion_detector.detect_emotions(image_resized)
    if emotions:
        dominant_emotion = max(emotions[0]['emotions'], key=emotions[0]['emotions'].get)
        all_emotions.append(dominant_emotion)
        for emotion, score in emotions[0]['emotions'].items():
            emotion_sums[emotion] += score

        # Determine the predominant emotion
        emotion_counts = Counter(all_emotions)
        predominant_emotion = emotion_counts.most_common(1)[0][0] if all_emotions else None

        return {
            "predominant_emotion": predominant_emotion,
            "emotions": emotion_sums
        }
    else:
        return None

def on_upload_change(change):
    global uploaded_file
    with output_area:
        clear_output()
        if upload_button.value:
            # Access the first element in the tuple and extract 'content'
            uploaded_file = upload_button.value[0]['content']
            print("File uploaded successfully. Please click 'Submit' to process.")
        else:
            print("Please upload a file.")

def on_submit_click(b):
    with output_area:
        if uploaded_file:
            clear_output()
            print("Processing the file...")
            result = analyze_image_emotions(uploaded_file)
            if result:
                print("Here is the result in JSON format:")
                print(json.dumps(result, indent=4))
            else:
                print("No emotions detected.")
        else:
            clear_output()
            print("Please upload a file first.")

# Register the callback for the upload and submit buttons
upload_button.observe(on_upload_change, names='value')
submit_button.on_click(on_submit_click)

# Display the widgets and the output area
display(upload_button, submit_button, output_area)


FileUpload(value=(), accept='image/*', description='Upload')

Button(description='Submit', style=ButtonStyle())

Output()

In [9]:
import cv2
from fer import FER
import numpy as np
from collections import Counter
from IPython.display import display, Image, clear_output
import PIL.Image
import io
import ipywidgets as widgets
import json

# Initialize FER emotion detector
emotion_detector = FER(mtcnn=True)

# Widgets for file upload and processing
upload_button = widgets.FileUpload(accept='image/*', multiple=False)
submit_button = widgets.Button(description='Submit')
output_area = widgets.Output()

uploaded_file_content = None
uploaded_file_name = None

def analyze_image_emotions(image_file_content):
    emotion_sums = {
        'angry': 0,
        'disgust': 0,
        'fear': 0,
        'happy': 0,
        'sad': 0,
        'surprise': 0,
        'neutral': 0
    }
    all_emotions = []

    # Convert the uploaded file content to an image
    image_stream = io.BytesIO(image_file_content)
    image_pil = PIL.Image.open(image_stream)
    image_np = np.array(image_pil)

    # Convert the image from RGB (PIL) to BGR (OpenCV)
    image_bgr = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)

    # Resize the image to a consistent size for analysis
    height, width = image_bgr.shape[:2]
    aspect_ratio = width / height
    new_height = 480
    new_width = int(new_height * aspect_ratio)
    image_resized = cv2.resize(image_bgr, (new_width, new_height))

    # Analyze the emotions in the image
    emotions = emotion_detector.detect_emotions(image_resized)
    if emotions:
        dominant_emotion = max(emotions[0]['emotions'], key=emotions[0]['emotions'].get)
        all_emotions.append(dominant_emotion)
        for emotion, score in emotions[0]['emotions'].items():
            emotion_sums[emotion] += score

        # Determine the predominant emotion
        emotion_counts = Counter(all_emotions)
        predominant_emotion = emotion_counts.most_common(1)[0][0] if all_emotions else None

        return {
            "filename": uploaded_file_name,
            "predominant_emotion": predominant_emotion,
            "emotions": emotion_sums
        }
    else:
        return {
            "filename": uploaded_file_name,
            "predominant_emotion": None,
            "emotions": emotion_sums
        }

def on_upload_change(change):
    global uploaded_file_content, uploaded_file_name
    with output_area:
        clear_output()
        if upload_button.value:
            # Access the first element in the tuple and extract 'content' and 'name'
            uploaded_file = upload_button.value[0]
            uploaded_file_content = uploaded_file['content']
            uploaded_file_name = uploaded_file['name']
            print(f"File '{uploaded_file_name}' uploaded successfully. Please click 'Submit' to process.")
        else:
            print("Please upload a file.")

def on_submit_click(b):
    with output_area:
        if uploaded_file_content:
            clear_output()
            print("Processing the file...")
            result = analyze_image_emotions(uploaded_file_content)
            print("Here is the result in JSON format:")
            print(json.dumps(result, indent=4))
        else:
            clear_output()
            print("Please upload a file first.")

# Register the callback for the upload and submit buttons
upload_button.observe(on_upload_change, names='value')
submit_button.on_click(on_submit_click)

# Display the widgets and the output area
display(upload_button, submit_button, output_area)


FileUpload(value=(), accept='image/*', description='Upload')

Button(description='Submit', style=ButtonStyle())

Output()

In [11]:
import cv2
from fer import FER
import numpy as np
from collections import Counter
from IPython.display import display, clear_output
import PIL.Image
import io
import ipywidgets as widgets
import json

# Initialize FER emotion detector
emotion_detector = FER(mtcnn=True)

# Widgets for file upload and processing
upload_button = widgets.FileUpload(accept='image/*', multiple=False)
submit_button = widgets.Button(description='Submit')
output_area = widgets.Output()

uploaded_file_content = None
uploaded_file_name = None

def analyze_image_emotions(image_file_content):
    emotion_sums = {
        'angry': 0,
        'disgust': 0,
        'fear': 0,
        'happy': 0,
        'sad': 0,
        'surprise': 0,
        'neutral': 0
    }
    all_emotions = []

    # Convert the uploaded file content to an image
    image_stream = io.BytesIO(image_file_content)
    image_pil = PIL.Image.open(image_stream)
    image_np = np.array(image_pil)

    # Convert the image from RGB (PIL) to BGR (OpenCV)
    image_bgr = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)

    # Resize the image to a consistent size for analysis
    height, width = image_bgr.shape[:2]
    aspect_ratio = width / height
    new_height = 480
    new_width = int(new_height * aspect_ratio)
    image_resized = cv2.resize(image_bgr, (new_width, new_height))

    # Analyze the emotions in the image
    emotions = emotion_detector.detect_emotions(image_resized)
    if emotions:
        dominant_emotion = max(emotions[0]['emotions'], key=emotions[0]['emotions'].get)
        all_emotions.append(dominant_emotion)

        # Determine the predominant emotion
        emotion_counts = Counter(all_emotions)
        predominant_emotion = emotion_counts.most_common(1)[0][0] if all_emotions else None

        return {
            "filename": uploaded_file_name,
            "predominant_emotion": predominant_emotion
        }
    else:
        return {
            "filename": uploaded_file_name,
            "predominant_emotion": None
        }

def on_upload_change(change):
    global uploaded_file_content, uploaded_file_name
    with output_area:
        clear_output()
        if upload_button.value:
            # Access the first element in the tuple directly
            uploaded_file = upload_button.value[0]
            uploaded_file_content = uploaded_file['content']
            uploaded_file_name = uploaded_file['name']
            print(f"File '{uploaded_file_name}' uploaded successfully. Please click 'Submit' to process.")
        else:
            print("Please upload a file.")

def on_submit_click(b):
    with output_area:
        if uploaded_file_content:
            clear_output()
            print("Processing the file...")
            result = analyze_image_emotions(uploaded_file_content)
            print("Here is the result in JSON format:")
            print(json.dumps(result, indent=4))
        else:
            clear_output()
            print("Please upload a file first.")

# Register the callback for the upload and submit buttons
upload_button.observe(on_upload_change, names='value')
submit_button.on_click(on_submit_click)

# Display the widgets and the output area
display(upload_button, submit_button, output_area)


FileUpload(value=(), accept='image/*', description='Upload')

Button(description='Submit', style=ButtonStyle())

Output()